### Processing SVI Census tract level data and HepVu death rates data into one dataframe
This is done for all the census tracts in the US

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

/Users/h6x/anaconda3/envs/TDA/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#import xlsx file as a dataframe
HepVu_df = pd.read_excel('./data/HepVu_County_Opioid_Indicators_05DEC22.xlsx')

In [3]:
HepVu_df.head()

,GEO ID,State Abbreviation,County Name,Opioid Prescription Rate 2020,Narcotic Overdose Mortality Rate 2014,Narcotic Overdose Mortality Rate 2015,Narcotic Overdose Mortality Rate 2016,Narcotic Overdose Mortality Rate 2017,Narcotic Overdose Mortality Rate 2018,Narcotic Overdose Mortality Rate 2019,Narcotic Overdose Mortality Rate 2020
0,1001,AL,Autauga County,98.3,8.2,8.8,10.7,9.8,10.9,9.2,11.6
1,1003,AL,Baldwin County,65.0,18.0,20.0,16.6,15.1,14.9,14.5,27.4
2,1005,AL,Barbour County,22.8,4.4,4.5,5.7,5.8,5.2,5.7,7.6
3,1007,AL,Bibb County,24.8,17.2,16.6,22.6,21.7,23.1,19.4,27.3
4,1009,AL,Blount County,22.8,18.6,18.9,22.7,27.0,19.9,20.3,24.2


In [4]:
HepVu_df.shape

(3221, 11)

#### Updating HepVu dataframe "GEO ID" column

###### GEOID column needs to change: insert "0" infront of 4 character values to match other dataframe

In [5]:
HepVu_df["GEO ID"] = HepVu_df["GEO ID"].astype(str)

In [6]:
# Check the length of each value in the 'FIPS' column, and if it's 4 characters, prepend '0'
HepVu_df['GEO ID'] = HepVu_df['GEO ID'].apply(lambda x: x.zfill(5) if len(x) == 4 else x)

In [7]:
HepVu_df["GEO ID"][123]

'05023'

#### Updating SVI "Year" dataframe

In [8]:
# SVI "Any year" dataset is a shapefile, so we can read it in using geopandas
SVI_df = gpd.read_file('./data/SVI data/census tract level/SVI2020_US_tract.gdb')

In [10]:
SVI_df.head(3)

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE,Shape_Length,Shape_Area,geometry
0,01,Alabama,AL,01001,Autauga,01001020100,"Census Tract 201, Autauga County, Alabama",3.793570,1941,390,...,1.8,0.0,1.8,6.6,5.1,0.0,1.8,0.150118,0.000949,"MULTIPOLYGON (((-86.50910 32.47349, -86.50577 ..."
1,01,Alabama,AL,01001,Autauga,01001020200,"Census Tract 202, Autauga County, Alabama",1.282174,1757,310,...,2.0,0.0,2.0,2.6,3.0,0.8,1.1,0.092328,0.000318,"MULTIPOLYGON (((-86.48093 32.48154, -86.47945 ..."
2,01,Alabama,AL,01001,Autauga,01001020300,"Census Tract 203, Autauga County, Alabama",2.065364,3694,570,...,0.9,0.0,0.9,1.8,2.3,0.0,0.9,0.100175,0.000516,"MULTIPOLYGON (((-86.47087 32.47573, -86.46964 ..."


Now we need to add narcotic death rates from HepVu to SVI dataframe \
For each census tract, related county death rates will be added for further computation.

In [11]:
SVI_HepVu_df = SVI_df.merge(HepVu_df[['GEO ID','Narcotic Overdose Mortality Rate 2020']], left_on="STCNTY", right_on="GEO ID")

In [12]:
# drop the 'GEO ID' column
SVI_HepVu_df.drop(columns=['GEO ID'], inplace=True)

In [13]:
SVI_HepVu_df.head(3)

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE,Shape_Length,Shape_Area,geometry,Narcotic Overdose Mortality Rate 2020
0,01,Alabama,AL,01001,Autauga,01001020100,"Census Tract 201, Autauga County, Alabama",3.793570,1941,390,...,0.0,1.8,6.6,5.1,0.0,1.8,0.150118,0.000949,"MULTIPOLYGON (((-86.50910 32.47349, -86.50577 ...",11.6
1,01,Alabama,AL,01001,Autauga,01001020200,"Census Tract 202, Autauga County, Alabama",1.282174,1757,310,...,0.0,2.0,2.6,3.0,0.8,1.1,0.092328,0.000318,"MULTIPOLYGON (((-86.48093 32.48154, -86.47945 ...",11.6
2,01,Alabama,AL,01001,Autauga,01001020300,"Census Tract 203, Autauga County, Alabama",2.065364,3694,570,...,0.0,0.9,1.8,2.3,0.0,0.9,0.100175,0.000516,"MULTIPOLYGON (((-86.47087 32.47573, -86.46964 ...",11.6


#### Saving dataframe to a geo file

In [14]:
# rename  'Opioid Prescription Rate'
SVI_HepVu_df.rename(columns = {'Narcotic Overdose Mortality Rate 2020':'NOD_Rate'}, inplace = True)

In [15]:
# Convert the DataFrame to a GeoDataFrame
SVI_HepVu_df = gpd.GeoDataFrame(SVI_HepVu_df, geometry='geometry')

In [16]:
# save SVI_HepVu_2018_df as a shapefile
SVI_HepVu_df.to_file("./data/processed data/SVI - census tract level/SVI 2020 with HepVu census tracts/SVI2020_US_census_with_opioid_indicators.shp")

/var/folders/j3/904q7dbj02j_jyjz7pvfgz384d40kw/T/ipykernel_70161/2493679348.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  SVI_HepVu_df.to_file("./data/processed data/SVI - census tract level/SVI 2020 with HepVu census tracts/SVI2020_US_census_with_opioid_indicators.shp")
